In [9]:
import pandas as pd
import glob


In [10]:
# display all rows and columns in the dataframes
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)


In [11]:
# merge the bike sharing demand csv files into one dataframe for Washington, D.C.
path = r'C:\Users\julia\Documents\Studium\Master\2. Semester\Master-Seminar Applied Econometrics - Data Science Basics\bike-sharing\datasets\bike-sharing'
all_files = glob.glob(path + "/*.csv")

df_list = []

for file in all_files:
    df_comp = pd.read_csv(file)
    df_list.append(df_comp)
    
df_final = pd.concat(df_list, axis=0, ignore_index=True)


In [12]:
# get overview of dataframe
print(df_final.shape)
df_final.head()


(13820361, 9)


,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,2389,2015-01-01 00:02:44,2015-01-01 00:42:33,31271,Constitution Ave & 2nd St NW/DOL,31254,15th & K St NW,W01140,Casual
1,2394,2015-01-01 00:02:46,2015-01-01 00:42:41,31271,Constitution Ave & 2nd St NW/DOL,31254,15th & K St NW,W00612,Casual
2,468,2015-01-01 00:04:32,2015-01-01 00:12:20,31204,20th & E St NW,31234,20th & O St NW / Dupont South,W01226,Member
3,348,2015-01-01 00:07:18,2015-01-01 00:13:06,31602,Park Rd & Holmead Pl NW,31117,15th & Euclid St NW,W20216,Member
4,980,2015-01-01 00:09:39,2015-01-01 00:26:00,31247,Jefferson Dr & 14th St SW,31241,Thomas Circle,W21005,Casual


In [8]:
# manual creation of dataframe containing information on holidays in Washington, D.C.
# source for this is https://dchr.dc.gov/page/holiday-schedules-2018, https://dchr.dc.gov/page/holiday-schedules-2016-and-2017
# https://dchr.dc.gov/page/holiday-schedules-2014-and-2015

df_holiday = pd.DataFrame(columns=['date', 'holiday'])

dates = ['2018-01-01', '2018-01-15', '2018-02-19', '2018-04-16',
         '2018-05-28', '2018-07-04', '2018-09-03', '2018-10-08',
         '2018-11-12', '2018-11-22', '2018-12-25', '2017-01-02',
         '2017-01-16', '2017-01-20', '2017-02-20', '2017-04-17',
         '2017-05-29', '2017-07-04', '2017-09-04', '2017-10-09',
         '2017-11-10', '2017-11-23', '2017-12-25', '2016-01-01',
         '2016-01-18', '2016-02-15', '2016-04-15', '2016-05-30',
         '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11',
         '2016-11-24', '2016-12-26', '2015-01-01', '2015-01-19',
         '2015-02-16', '2015-04-16', '2015-05-25', '2015-07-03',
         '2015-09-07', '2015-10-12', '2015-11-11', '2015-11-26',
         '2015-12-25']

df_holiday['date'] = dates
df_holiday['holiday'] = 1
df_holiday.head()


,date,holiday
0,2018-01-01,1
1,2018-01-15,1
2,2018-02-19,1
3,2018-04-16,1
4,2018-05-28,1


In [12]:
# read in the weather data for Washington, D.C.
path = r'C:\Users\julia\Documents\Studium\Master\2. Semester\Master-Seminar Applied Econometrics - Data Science Basics\bike-sharing\datasets\weather-data'
all_files = glob.glob(path + "/*.csv")

df_list_weather = []

for file in all_files:
    df_comp_weather = pd.read_csv(file)
    df_list_weather.append(df_comp_weather)
    
df_final_weather = pd.concat(df_list_weather, axis=0, ignore_index=True)


C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,11,15,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,11,15,25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
# number of rows and columns in weather dataframe
df_final_weather.shape


(136180, 30)

In [21]:
# overview of weather dataframe
df_final_weather[df_final_weather['DATE'] == '2017-01-01']

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,AWND_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,TAVG,TAVG_ATTRIBUTES,WT01,WT01_ATTRIBUTES,WT02,WT02_ATTRIBUTES,WT03,WT03_ATTRIBUTES,WT04,WT04_ATTRIBUTES,WT05,WT05_ATTRIBUTES,WT06,WT06_ATTRIBUTES,WT08,WT08_ATTRIBUTES,WT09,WT09_ATTRIBUTES,WT11,WT11_ATTRIBUTES
0,US1MDMG0109,"SILVER SPRING 2.1 NNE, MD US",39.039861,-77.004083,103.3,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
852,US1MDMG0106,"SILVER SPRING 6.4 NNE, MD US",39.080732,-77.008126,125.0,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1579,US1MDAA0043,"DEALE 0.9 SE, MD US",38.783953,-76.536154,2.1,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649,US1VAFX0064,"FALLS CHURCH 1.6 W, VA US",38.887500,-77.204100,141.4,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4130,US1VAFX0060,"OAKTON 0.4 E, VA US",38.892197,-77.293236,113.7,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4410,US1MDMG0066,"BETHESDA 2.0 WNW, MD US",38.998058,-77.155051,68.3,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5090,US1MDPG0001,"BOWIE 0.5 E, MD US",38.960100,-76.727500,40.8,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6341,US1MDMG0065,"POOLESVILLE 0.4 SE, MD US",39.140067,-77.404641,113.4,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7065,US1VAFX0063,"ALEXANDRIA 5.6 SSW, VA US",38.731065,-77.088951,14.9,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7787,US1MDCV0018,"CHESAPEAKE BEACH 0.1 SSE, MD US",38.685683,-76.534533,16.5,2017-01-01,NaN,NaN,0.0,",,N",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
